# Main project description and setup

In [2]:
import sklearn.datasets
import torch
from torch import nn

<br><br><br><br><br>

## Project 1: classify jets with a neural network

### Step 1: Download and understand the data

https://openml.org/search?type=data&status=active&id=42468

In [5]:
hls4ml_lhc_jets_hlf = sklearn.datasets.fetch_openml("hls4ml_lhc_jets_hlf")

features, targets = hls4ml_lhc_jets_hlf["data"], hls4ml_lhc_jets_hlf["target"]

In [6]:
features

,zlogz,c1_b0_mmdt,c1_b1_mmdt,c1_b2_mmdt,c2_b1_mmdt,c2_b2_mmdt,d2_b1_mmdt,d2_b2_mmdt,d2_a1_b1_mmdt,d2_a1_b2_mmdt,m2_b1_mmdt,m2_b2_mmdt,n2_b1_mmdt,n2_b2_mmdt,mass_mmdt,multiplicity
0,-2.935125,0.383155,0.005126,0.000084,0.009070,0.000179,1.769445,2.123898,1.769445,0.308185,0.135687,0.083278,0.412136,0.299058,8.926882,75.0
1,-1.927335,0.270699,0.001585,0.000011,0.003232,0.000029,2.038834,2.563099,2.038834,0.211886,0.063729,0.036310,0.310217,0.226661,3.886512,31.0
2,-3.112147,0.458171,0.097914,0.028588,0.124278,0.038487,1.269254,1.346238,1.269254,0.246488,0.115636,0.079094,0.357559,0.289220,162.144669,61.0
3,-2.666515,0.437068,0.049122,0.007978,0.047477,0.004802,0.966505,0.601864,0.966505,0.160756,0.082196,0.033311,0.238871,0.094516,91.258934,39.0
4,-2.484843,0.428981,0.041786,0.006110,0.023066,0.001123,0.552002,0.183821,0.552002,0.084338,0.048006,0.014450,0.141906,0.036665,79.725777,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
829995,-3.575320,0.473246,0.040693,0.005605,0.053711,0.004402,1.319914,0.785488,1.319914,0.211968,0.106151,0.037546,0.315867,0.123637,72.537308,71.0
829996,-2.408292,0.429539,0.040022,0.005620,0.020352,0.000804,0.508506,0.143106,0.508506,0.077383,0.043065,0.011398,0.131738,0.028787,77.263367,30.0
829997,-3.338864,0.467011,0.075235,0.017644,0.097954,0.022681,1.301970,1.285501,1.301970,0.236583,0.110919,0.068624,0.307230,0.183485,136.165955,72.0
829998,-1.535967,0.335411,0.002537,0.000021,0.002692,0.000017,1.061160,0.797847,1.061160,0.175014,0.086063,0.048476,0.271106,0.161818,4.660848,11.0


In [8]:
features.describe()

,zlogz,c1_b0_mmdt,c1_b1_mmdt,c1_b2_mmdt,c2_b1_mmdt,c2_b2_mmdt,d2_b1_mmdt,d2_b2_mmdt,d2_a1_b1_mmdt,d2_a1_b2_mmdt,m2_b1_mmdt,m2_b2_mmdt,n2_b1_mmdt,n2_b2_mmdt,mass_mmdt,multiplicity
count,830000.000000,830000.000000,830000.000000,8.300000e+05,830000.000000,8.300000e+05,830000.000000,830000.000000,830000.000000,830000.000000,830000.000000,830000.000000,830000.000000,830000.000000,830000.000000,830000.000000
mean,-2.865343,0.433322,0.037766,7.995166e-03,0.045608,7.609470e-03,1.295784,1.083618,1.295784,0.190380,0.090024,0.042460,0.281169,0.143915,75.153610,51.887834
std,0.580389,0.055448,0.029154,9.402567e-03,0.038657,1.217365e-02,0.458041,0.730066,0.458041,0.075417,0.036523,0.026396,0.084556,0.080461,55.612557,21.677036
min,-4.759511,0.091104,0.000073,4.472011e-08,0.000002,1.472518e-10,0.005866,0.000156,0.005866,0.000213,0.000077,0.000002,0.000643,0.000018,0.113449,6.000000
25%,-3.283773,0.419295,0.009977,3.371321e-04,0.015352,4.735599e-04,0.976546,0.485602,0.976546,0.125212,0.059285,0.018935,0.213851,0.071025,19.084184,36.000000
50%,-2.909453,0.452219,0.037919,5.950152e-03,0.036848,2.501090e-03,1.278506,0.983084,1.278506,0.192994,0.089061,0.038755,0.292299,0.139280,80.106373,48.000000
75%,-2.493677,0.468801,0.048510,8.193400e-03,0.062181,7.816279e-03,1.559999,1.505659,1.559999,0.251016,0.118213,0.062612,0.350496,0.210668,93.843903,64.000000
max,-0.438996,0.493779,0.165237,7.122659e-02,0.219034,1.079140e-01,3.968144,6.408456,3.968144,0.366573,0.187837,0.137693,0.449523,0.337616,573.616516,212.000000


In [7]:
targets

0         g
1         w
2         t
3         z
4         w
         ..
829995    z
829996    w
829997    t
829998    q
829999    g
Name: class, Length: 830000, dtype: category
Categories (5, object): ['g', 'q', 't', 'w', 'z']

<img src="../img/JetDataset.png" width="429">

<br><br><br><br><br>

### Step 2: Split the data into training, validation, and test samples

* 80% of the data the training sample, which the optimizer will use in its fits
* 10% of the data in the validation sample, which you will look at while developing the model
* 10% of the data in the test sample, which you should not look at until you're done

These data are supposed to be Independent and Identically Distributed (IID), but just in case there are any beginning-of-dataset, end-of-dataset biases, sample them randomly.

Note that Scikit-Learn has a [sklearn.model_selection.train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) and PyTorch has a [torch.utils.data.random_split](https://pytorch.org/docs/stable/data.html#torch.utils.data.random_split).

<br><br><br><br><br>

### Step 3: Build a classifier neural network

It should

* take the 16 numerical features as input
* have 3 hidden layers with 32 neurons each
* return probabilities for the 5 output categories

Use any tools you have to improve the quality of the model.

<br><br><br><br><br>

### Step 4: Monitor the loss function